<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">

# **Liquidation Simulation in Halalend Protocol**
#### Within the Halalend lending protocol, maintaining the stability and security of loans is paramount. This section simulates the protocol's automated system designed to manage liquidations. Liquidation occurs when the value of collateral falls below a certain threshold, necessitating action to mitigate potential losses. We'll delve into how the protocol identifies loans at risk and executes liquidation to safeguard both the borrower's and lender's interests.

## Assuming we have the following **Protocol Parameters:**

</div>


In [59]:
// Collateral and Loan Settings
let minimum_collateral_ratio = 2.0  // Minimum Collateral Ratio for ADA as Asset A
let minimum_collateral_share_assetb = 0.1 // Minimum percentage required for collateral in Asset B
let liquidation_threshold = 1.5  // Collateral Ratio at which liquidation starts 

// Loan Settings
let minimum_loan_amount = 100.0  // USD, minimum amount one can borrow
let maximum_loan_term_in_milliseconds = 1_209_600_000  // Maximum term for a loan in milliseconds

// Financial and Risk Management Settings
let liquidation_fee = 10.0  // USD, fee charged upon liquidation
let protocol_usage_fee = 5.0  // USD, fee for using the protocol per loan operation

<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">


# **Liquidation Calculations Explained**

---

### **Current Collateral Ratio Calculation**

##### The Current *Collateral Ratio* is a critical measure in assessing the health of a loan. It's calculated by summing the value of all collateral assets (Asset A and Asset B) and dividing by the total loan amount, factoring in the respective exchange rates for each asset and the loan currency.

```
calculate_collateral_ratio a au b bu c cu = ((a * au) + (b * bu)) / (c * cu)
```

#### Here, *a* and *b* represent the quantities of Asset A and Asset B, while *au* and *bu* are their respective unit values in USD. *c* is the loan amount, and *cu* is the currency unit value (typically 1 if the loan is in USD).
---

### **Liquidation Eligibility Check**

##### The protocol's automated system periodically assesses each loan to determine if it's eligible for liquidation. This involves two key checks:

- *Loan Term Expiry:* If the elapsed time since the loan was issued exceeds the agreed loan term, the loan is considered expired and eligible for liquidation.
- *Collateral Value Drop*: If the current collateral ratio falls below the predefined liquidation threshold, it indicates that the collateral no longer provides adequate security for the loan due to a decrease in its market value.

```
is_liquidatable a au b bu c cu loan_term elapsed_time =
    let is_expired = elapsed_time > loan_term
    let current_collateral_ratio = calculate_collateral_ratio a au b bu c cu
    let is_below_liquidation_threshold = current_collateral_ratio < liquidationThreshold

    printf "Current Collateral Ratio: %f\n" current_collateral_ratio
    printf "Is Expired: %b\n" is_expired
    printf "Is Below Liquidation Threshold: %b\n" is_below_liquidation_threshold
    printf "Liquidation Threshold: %f\n\n" liquidationThreshold

    is_expired || is_below_liquidation_threshold
```

#### Here:
#### - *elapsed_time:* The time that has passed since the loan was taken out.
#### - *loan_term:* The agreed duration of the loan.

#### This function prints out the current collateral ratio, whether the loan term has expired, if the collateral ratio is below the liquidation threshold, and what the liquidation threshold is. If either condition is met, the loan is marked for liquidation.
---
</div>

In [60]:
let calculate_collateral_ratio a au b bu c cu = ((a * au) + (b * bu)) / (c * cu)

// check if a loan position is for liquidation
let is_liquidatable a au b bu c cu loan_term elapsed_time =
    let is_expired = elapsed_time > loan_term
    let current_collateral_ratio = calculate_collateral_ratio a au b bu c cu
    let is_below_liquidation_threshold = current_collateral_ratio < liquidation_threshold

    printf "Current Collateral Ratio: %f\n" current_collateral_ratio
    printf "Is Expired: %b\n" is_expired
    printf "Is Below Liquidation Threshold: %b\n" is_below_liquidation_threshold
    printf "Liquidation Threshold: %f\n\n" liquidation_threshold

    is_expired || is_below_liquidation_threshold
    

<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">

## <span style="color: #CCCCCC;">Evaluation for Liquidation</span>

#### To assess whether a loan position should be liquidated, the protocol follows these steps:

---

- *Calculate the Current Collateral Value:*  Using the provided collateral amounts and exchange rates, the total value of the collateral in USD is computed as current_collateral_value.
- *Determine the Current Collateral Ratio (CR):* The current CR is calculated by dividing current_collateral_value by loan_amount_usd, taking into account the exchange rates.
- *Check Against the Liquidation Threshold:* The current CR is compared to the liquidation threshold, represented as liquidation_threshold. If the current CR falls below this threshold or if the loan term has expired (elapsed_time >= loan_term_in_milliseconds), the protocol initiates the liquidation process.

#### Based on the calculated CR and the evaluation criteria, the protocol determines whether the loan position is eligible for liquidation. The outcome may vary depending on the loan details and the current market conditions. When a loan is liquidated, the protocol collects the loan amount from the locked collateral value plus the liquidation fee. Any remaining collateral will be sent back to the borrower.

</div>




## <span style="color: #CCCCCC;">Loan Position Details</span>

In [61]:
// amount to borrow and exchange rate
let loan_amount = 120.0
let loan_exchange_rate = 1.0

// amount of collateral and exchange rate
let asset_a_collateral = 350.0
let asset_a_exchange_rate = 0.45
let asset_b_collateral = 500.0
let asset_b_exchange_rate = 0.03

// loan term
let loan_term = 1_209_600_000

// elapsed time
let elapsed_time = 1_209_600_000


## <span style="color: #CCCCCC;">Liquidation Check</span>

In [62]:
// calculate usd value
let asset_a_usd = asset_a_collateral * asset_a_exchange_rate
let asset_b_usd = asset_b_collateral * asset_b_exchange_rate
let total_collateral = asset_a_usd + asset_b_usd
let loan_amount_usd = loan_amount * loan_exchange_rate
let loan_to_value = (loan_amount_usd / total_collateral) * 100.0

printf "LOAN DETAILS\n"
printf "Loan Amount: %f USD\n" loan_amount_usd
printf "Asset A Collateral: %f USD\n" asset_a_usd
printf "Asset B Collateral: %f USD\n" asset_b_usd
printf "Total Collateral: %f USD\n" total_collateral
printf "Loan to Value: %f %%\n\n" loan_to_value

let can_be_liquidated = is_liquidatable asset_a_collateral asset_a_exchange_rate asset_b_collateral asset_b_exchange_rate loan_amount loan_exchange_rate loan_term elapsed_time

printf "Can be liquidated: %b\n" can_be_liquidated

LOAN DETAILS
Loan Amount: 120.000000 USD
Asset A Collateral: 157.500000 USD
Asset B Collateral: 15.000000 USD
Total Collateral: 172.500000 USD
Loan to Value: 69.565217 %

Current Collateral Ratio: 1.437500
Is Expired: false
Is Below Liquidation Threshold: true
Liquidation Threshold: 1.500000

Can be liquidated: true
